# Pipeline for Production
So the approach that I used (for simplicity) that I get the data from dataframe. based on a real world scenario it could be upload or pulled from or some other services then we can change the code accordingly 

In [ ]:
import joblib
# Load the model
loaded_model = joblib.load('../models/random_forest_model.pkl')

# Use the loaded model to make predictions
# predictions = loaded_model.predict(X_test)

In [1]:
def extract_features(df):
     """
     Extracts features from raw sensor data at the session level.
     
     Parameters:
          df (pd.DataFrame): The sensor data containing columns ['id', 'time', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'side']
          time could come also as index for the dataframe as well
     
     Returns:
          pd.DataFrame: Processed session-level features with step counts.
     """
     # Ensure 'time' is a datetime type
     if df.get("time") is not None:
          if not pd.api.types.is_datetime64_any_dtype(df['time']):
               df["time"] = pd.to_datetime(df["time"])
          df.set_index('time', inplace=True)
          df = df.sort_index()
          
     if not isinstance(df.index, pd.DatetimeIndex):
          df.index = pd.to_datetime(df.index)
     


     # Extract time-based features
     session_metadata["session_duration"] = (df.index.max() - df.index.min()).total_seconds()
     session_metadata['num_measurements'] = len(df)
     session_metadata['start_time'] = df.index.min()
     session_metadata['end_time'] = df.index.max()
     # maybe not need
     # if df.get("time_diff") is None:
     #      df["time_diff"] = df.index.to_series().diff().dt.total_seconds()
     
     
     # Aggregate sensor statistics per session
     session_features = df.groupby("id").agg({
          "ax": ["mean", "std", "min", "max"],
          "ay": ["mean", "std", "min", "max"],
          "az": ["mean", "std", "min", "max"],
          "gx": ["mean", "std", "min", "max"],
          "gy": ["mean", "std", "min", "max"],
          "gz": ["mean", "std", "min", "max"]
          # "time_diff": ["sum", "count"]  # Sum gives session duration
     }).reset_index()

     # Flatten MultiIndex columns
     session_features.columns = ["_".join(col).strip() for col in session_features.columns.values if col != "id"]
     # session_features = session_features.rename(columns={"id_": "id"})

     # Function to count steps using peaks in 'az'
     def count_peaks(series):
          peaks, _ = find_peaks(series, height=-1.66)  
          return len(peaks)

     step_counts = df.groupby(["id", "side"])["az"].apply(count_peaks).unstack(fill_value=0)
     step_counts.columns = ["left_steps", "right_steps"]

     # merge all data
     final_df = session_metadata.merge(session_features, on="id").merge(step_counts, on="id")

     return final_df

In [ ]:
# Generate Predictions for a New Session
def predict_steps(new_session_data):
    new_session_features = extract_features(new_session_data)  

    # Extract only model input features
    X_new = new_session_features.drop(columns=["id", "start_time", "end_time", "left_steps", "right_steps"])

    # Predict step counts
    predicted_steps = model.predict(X_new)

    # Convert predictions to a structured output
    new_session_features["left_steps_pred"] = predicted_steps[:, 0]
    new_session_features["right_steps_pred"] = predicted_steps[:, 1]

    # Final structured output
    output = new_session_features[["id", "start_time", "end_time", "left_steps_pred", "right_steps_pred"]]
    output.rename(columns={"left_steps_pred": "left_steps", "right_steps_pred": "right_steps"}, inplace=True)

    return output.to_dict(orient="records")

# Example Prediction
example_session = df[df["id"] == "0373xrf1eaJoc8IcE6Gc"]
predicted_steps = predict_steps(example_session)
print(f"Predicted Steps: {predicted_steps}")